In [2]:
import numpy as np
import pymatgen
from pymatgen.ext.matproj import MPRester
import pandas as pd
from pymatgen.core import Structure, Composition  # <-- import Structure and Composition classes from pymatgen.core
from pymatgen.io.cif import CifParser
import pandas as pd
import matminer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
#from sklearn.linear_model import LinearRegression
import matminer
from matminer.datasets import load_dataset


In [3]:
flla_data = load_dataset('matbench_mp_gap')

In [14]:
from pymatgen.core.structure import Structure
import pandas as pd
dff2=pd.DataFrame(flla_data)

In [74]:
#df2=df2.drop('Unnamed: 0',axis=1)

In [15]:
cotst=pd.DataFrame(dff2)

In [9]:
pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",100)

In [16]:
cotst = cotst.dropna()
cotst

,structure,gap pbe
0,"[[-0.00812638 0.02476014 -0.01698117] K, [-0....",1.3322
1,"[[0. 1.78463544 1.78463544] Cr, [1.784...",0.0000
2,"[[-2.13764909 -2.12540569 -2.14704542] Cs, [-6...",0.0000
3,"[[0. 0. 0.] Si, [ 4.55195829 4.55195829 -4.55...",0.4113
4,"[[0. 2.655 2.655] Ca, [2.655 0. 2.655] C...",0.3514
...,...,...
106108,"[[ 2.91058377 3.61215869 -0.19100541] Ca, [-0...",1.1354
106109,"[[0.07215014 3.75835129 1.91249744] Ta, [2.014...",2.7274
106110,"[[0.99954964 0.70129827 4.70919163] Mg, [ 0.87...",2.8860
106111,"[[0.99298226 0.71146045 4.70710628] Zn, [ 0.86...",2.2330


In [ ]:
cotst = cotst.iloc[:len(cotst)//10]
#half2 = df.iloc[len(df)//2:]

    



In [19]:
cotst

,structure,gap pbe
0,"[[-0.00812638 0.02476014 -0.01698117] K, [-0....",1.3322
1,"[[0. 1.78463544 1.78463544] Cr, [1.784...",0.0000
2,"[[-2.13764909 -2.12540569 -2.14704542] Cs, [-6...",0.0000
3,"[[0. 0. 0.] Si, [ 4.55195829 4.55195829 -4.55...",0.4113
4,"[[0. 2.655 2.655] Ca, [2.655 0. 2.655] C...",0.3514
...,...,...
53051,"[[-2.32197021 3.83490794 3.24285397] Li, [-0...",2.4664
53052,"[[ 1.94393673 -5.12476594 3.37287661] W, [2.7...",0.0000
53053,"[[-7.16990887 -0.01102514 2.40216949] Ca, [-9...",2.9678
53054,"[[2.59721101 2.45197407 1.43088351] Li, [7.766...",0.5545


In [20]:
from matminer.featurizers.conversions import StructureToComposition
comp=StructureToComposition()
cotst=comp.featurize_dataframe(cotst,col_id='structure',ignore_errors=True)

StructureToComposition:   0%|          | 0/53056 [00:00<?, ?it/s]

,structure,gap pbe,composition
0,"[[-0.00812638 0.02476014 -0.01698117] K, [-0....",1.3322,"(K, Mn, O)"
1,"[[0. 1.78463544 1.78463544] Cr, [1.784...",0.0000,"(Cr, Ni)"
2,"[[-2.13764909 -2.12540569 -2.14704542] Cs, [-6...",0.0000,"(Cs, Rb, As)"
3,"[[0. 0. 0.] Si, [ 4.55195829 4.55195829 -4.55...",0.4113,"(Si, Sn)"
4,"[[0. 2.655 2.655] Ca, [2.655 0. 2.655] C...",0.3514,"(Ca, P)"
...,...,...,...
106108,"[[ 2.91058377 3.61215869 -0.19100541] Ca, [-0...",1.1354,"(Ca, Cu, F)"
106109,"[[0.07215014 3.75835129 1.91249744] Ta, [2.014...",2.7274,"(Ta, O)"
106110,"[[0.99954964 0.70129827 4.70919163] Mg, [ 0.87...",2.8860,"(Mg, Mo, O)"
106111,"[[0.99298226 0.71146045 4.70710628] Zn, [ 0.86...",2.2330,"(Zn, Mo, O)"


In [ ]:
from matminer.featurizers.composition import ElementFraction
ef = ElementFraction()
feat1=ef.featurize_dataframe(cotst,col_id='composition')

from matminer.featurizers.structure.bonding import StructuralHeterogeneity
cou=StructuralHeterogeneity()
cou.fit(cotst['structure'])
feat2=cou.featurize_dataframe(cotst,col_id='structure',ignore_errors=True)

from matminer.featurizers.conversions import CompositionToOxidComposition
oxy= CompositionToOxidComposition()
feat3=oxy.featurize_dataframe(cotst,col_id='composition',ignore_errors=True)


from matminer.featurizers.composition.ion import OxidationStates
oxida=OxidationStates()
feat4=oxida.featurize_dataframe(feat3,col_id='composition_oxid',ignore_errors=True)


from matminer.featurizers.composition.ion import IonProperty
ostafea= IonProperty()
feat5=ostafea.featurize_dataframe(cotst,col_id='composition',ignore_errors=True)

ElementFraction:   0%|          | 0/53056 [00:00<?, ?it/s]

StructuralHeterogeneity:   0%|          | 0/53056 [00:00<?, ?it/s]

In [ ]:
from matminer.featurizers.composition.orbital import ValenceOrbital
val=ValenceOrbital()
feat6=val.featurize_dataframe(cotst,col_id='composition',ignore_errors=True)

from matminer.featurizers.composition.element import TMetalFraction
met=TMetalFraction()
feat7=met.featurize_dataframe(cotst,col_id='composition',ignore_errors=True)


from matminer.featurizers.composition.element import Stoichiometry
stoich = Stoichiometry()
feat8=stoich.featurize_dataframe(cotst,col_id='composition',ignore_errors=True)


In [ ]:

from matminer.featurizers.composition.orbital import AtomicOrbitals
Ato=AtomicOrbitals()
feat9=Ato.featurize_dataframe(cotst,col_id='composition',ignore_errors=True)



from matminer.featurizers.structure.order import DensityFeatures
Den=DensityFeatures()
Den.fit(cotst['structure'])
feat11=Den.featurize_dataframe(cotst,col_id='structure',ignore_errors=True)


In [ ]:
from matminer.featurizers.structure.order import ChemicalOrdering
chem=ChemicalOrdering()
chem.fit(cotst['structure'])
feat12=chem.featurize_dataframe(cotst,col_id='structure',ignore_errors=True)


from matminer.featurizers.composition.element import BandCenter
band=BandCenter()
#band.fit(cotst['composition'])
feat10=band.featurize_dataframe(df=cotst,col_id='composition',ignore_errors=True)

In [ ]:
feat9=feat9.drop(['HOMO_element','LUMO_element','HOMO_character','LUMO_character'],axis=1)

feat1=feat1.drop(['structure','bandgap','composition'],axis=1)
feat2=feat2.drop(['structure','bandgap','composition'],axis=1)
#feat3=feat3.drop(['structure','bandgap','composition','composition_oxid'],axis=1)
feat4=feat4.drop(['composition_oxid','structure','bandgap','composition',],axis=1)
feat5=feat5.drop(['compound possible','structure','bandgap','composition'],axis=1)
feat6=feat6.drop(['structure','bandgap','composition'],axis=1)
feat7=feat7.drop(['structure','bandgap','composition'],axis=1)
feat8=feat8.drop(['structure','bandgap','composition'],axis=1)
feat9=feat9.drop(['structure','bandgap','composition'],axis=1)
feat10=feat10.drop(['structure','bandgap','composition'],axis=1)
feat11=feat11.drop(['structure','bandgap','composition'],axis=1)
feat12=feat12.drop(['structure','composition','bandgap'],axis=1)



In [ ]:
result=feat1.join(feat2)
result=result.join(feat4)
result=result.join(feat5)
result=result.join(feat6)
result=result.join(feat7)
result=result.join(feat8)
result=result.join(feat9)
result=result.join(feat10)
result=result.join(feat11)
result=result.join(feat12)
#result=result.join(feat12)
#result=result.join(feat13)

In [ ]:
# Create empty DataFrames to store features and target
Q = pd.DataFrame(columns=['mean_atomic_numbers', 'max_atomic_numbers', 'min_atomic_numbers', 'std_atomic_numbers',
                          'a_parameters', 'b_parameters', 'c_parameters', 'alpha_parameters', 'beta_parameters',
                          'gamma_parameters', 'mean_distance_matrix', 'max_distance_numbers', 'min_distance_numbers',
                          'std_distance_numbers'])
#y = pd.DataFrame(columns=['formation_energy_per_atom'])

def extract_features(structure):
    # Atomic number features
    atomic_numbers = [site.specie.number for site in structure.sites]
    mean_atomic_numbers = np.mean(atomic_numbers)
    max_atomic_numbers = np.max(atomic_numbers)
    min_atomic_numbers = np.min(atomic_numbers)
    std_atomic_numbers = np.std(atomic_numbers)

    # Lattice parameter features
    lattice = structure.lattice
    a_parameters = lattice.a
    b_parameters = lattice.b
    c_parameters = lattice.c
    alpha_parameters = lattice.alpha
    beta_parameters = lattice.beta
    gamma_parameters = lattice.gamma

    # Distance matrix features
    distance_matrix = structure.distance_matrix
    mean_distance_matrix = np.mean(distance_matrix)
    max_distance_numbers = np.max(distance_matrix)
    min_distance_numbers = np.min(distance_matrix)
    std_distance_numbers = np.std(distance_matrix)


    return {
        'mean_atomic_numbers': mean_atomic_numbers,
        'max_atomic_numbers': max_atomic_numbers,
        'min_atomic_numbers': min_atomic_numbers,
        'std_atomic_numbers': std_atomic_numbers,
        'a_parameters': a_parameters,
        'b_parameters': b_parameters,
        'c_parameters': c_parameters,
        'alpha_parameters': alpha_parameters,
        'beta_parameters': beta_parameters,
        'gamma_parameters': gamma_parameters,
        'mean_distance_matrix': mean_distance_matrix,
        'max_distance_numbers': max_distance_numbers,
        'min_distance_numbers': min_distance_numbers,
        'std_distance_numbers': std_distance_numbers
    }

for index, row in cotst.iterrows():
    #Extract the structure and formation energy for the current material
    structure = row['structure']
    bandgap= row['bandgap']

    #Extract the features from the structure
    features = extract_features(structure)

    #Add the features and formation energy to the X and y DataFrames
    Q = Q.append(features, ignore_index=True)
   # y = y.append({'formation_energy_per_atom': formation_energy}, ignore_index=True)

#pd.set_option('display.max_columns',None)
result=result.join(Q)
print(result)
#print(result)



result.to_csv("hlflakhdgx.csv",index=False)
